In [35]:
import pandas as pd
import ast

In [36]:
df=pd.read_excel("jaipur_cars.xlsx")


In [37]:

df['new_car_detail'] = df['new_car_detail'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
new_car_details_df = pd.json_normalize(df['new_car_detail'])

In [38]:
new_car_details_df

,it,ft,bt,km,transmission,ownerNo,owner,oem,model,modelYear,centralVariantId,variantName,price,priceActual,priceSaving,priceFixedText,trendingText.imgUrl,trendingText.heading,trendingText.desc
0,0,Diesel,Hatchback,"1,20,000",Manual,2,2nd Owner,Hyundai,Hyundai i20,2015,1487,Asta 1.4 CRDi,₹ 5.10 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days
1,0,Petrol,Hatchback,"66,951",Manual,1,1st Owner,Maruti,Maruti Swift,2012,10403,VXI,₹ 3.81 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days
2,0,Petrol,Hatchback,"80,000",Automatic,2,2nd Owner,Maruti,Maruti Celerio,2016,3982,VXI AT,₹ 3.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days
3,0,Petrol,Hatchback,"44,392",Manual,1,1st Owner,Hyundai,Hyundai Grand i10,2017,5602,1.2 Kappa Magna BSIV,₹ 4.20 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days
4,0,Petrol,Hatchback,"40,000",Automatic,1,1st Owner,Maruti,Maruti Wagon R,2016,4319,AMT VXI,₹ 3.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1115,0,Petrol,Hatchback,"1,20,000",Manual,1,1st Owner,Chevrolet,Chevrolet Beat,2011,438,LT,₹ 1.08 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days
1116,0,Petrol,Hatchback,"70,000",Manual,1,1st Owner,Maruti,Maruti Alto,2007,3917,STD,₹ 1.05 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days
1117,0,Petrol,SUV,"20,000",Manual,1,1st Owner,Renault,Renault Kiger,2022,8350,RXZ,₹ 8.50 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days
1118,0,Petrol,Hatchback,"40,000",Manual,1,1st Owner,Maruti,Maruti Alto,2009,3904,Std,₹ 1 Lakh,,,None,https://stimg.cardekho.com/used-cars/common/icons/trending.svg,Trending Car!,High chances of sale in next 6 days


In [42]:


# Extracting 'top' key values into separate columns
def extract_keys(dict_list):
    result = {}
    for item in dict_list:
        result[item['key']] = item['value']
    return result

# Apply function and expand the dictionary into columns
df['new_car_overview'] = df['new_car_overview'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
car_overview = df["new_car_overview"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)


In [43]:
car_overview

,Registration Year,Insurance Validity,Fuel Type,Seats,Kms Driven,RTO,Ownership,Engine Displacement,Transmission,Year of Manufacture
0,2015,Third Party insurance,Diesel,5 Seats,"1,20,000 Kms",RJ13,Second Owner,1396 cc,Manual,2015.0
1,Jan 2012,Third Party,Petrol,5 Seats,"66,951 Kms",RJ14,First Owner,1197 cc,Manual,2012.0
2,2016,Third Party insurance,Petrol,5 Seats,"80,000 Kms",RJ14,Second Owner,998 cc,Automatic,2016.0
3,Apr 2017,Third Party insurance,Petrol,5 Seats,"44,392 Kms",RJ14,First Owner,1197 cc,Manual,2017.0
4,2016,Third Party insurance,Petrol,5 Seats,"40,000 Kms",RJ14,First Owner,998 cc,Automatic,2016.0
...,...,...,...,...,...,...,...,...,...,...
1115,2011,Third Party insurance,Petrol,5 Seats,"1,20,000 Kms",RJ14,First Owner,1199 cc,Manual,2011.0
1116,2007,Third Party insurance,Petrol,5 Seats,"70,000 Kms",RJ30,First Owner,796 cc,Manual,2007.0
1117,2022,Third Party insurance,Petrol,5 Seats,"20,000 Kms",RJ45,First Owner,999 cc,Manual,2022.0
1118,2009,Third Party insurance,Petrol,5 Seats,"40,000 Kms",NaN,First Owner,796 cc,Manual,2009.0


In [30]:
df['new_car_feature'] = df['new_car_feature'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

In [32]:
car_features = df["new_car_feature"].apply(lambda x: extract_keys(x['top'])).apply(pd.Series)

KeyError: 'key'

In [34]:
columns_data = {}

# Extract the subheading and list of values
for section in df['new_car_feature']:
    subheading = section['subHeading']
    values_list = [item['value'] for item in section['list']]
    columns_data[subheading] = values_list

KeyError: 'subHeading'